# The battle of Neighborhoods in Athens

## Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#! conda install -c conda-forge folium

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
#Read the data
df_data_0 = pd.read_csv("Geo_Athens.csv")
df_data_0.head()

,Neighborhood,Latitude,Longitude
0,Akademia Platonos,37.991368,23.708381
1,Ampelokipoi,37.989403,23.761144
2,Ano Kypseli,38.005780,23.745380
3,Ano Petralona,37.970467,23.712938
4,Exarcheia,37.985992,23.735541


In [3]:
# Get the latitude and longitude values of Athens.
address = 'Athens, Greece'

geolocator = Nominatim(user_agent = "athens_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Athens are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Athens are 37.9839412, 23.7283052.


In [4]:
# Create map of Athens using latitude and longitude values
map_data = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_data_0['Latitude'], df_data_0['Longitude'], df_data_0['Neighborhood']):
    label = '{}'.format(df_data_0)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_data)  
    
map_data

In [39]:
CLIENT_ID = 'YOUR_CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'YOUR_CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOUR_CLIENT_ID
CLIENT_SECRET:YOUR_CLIENT_SECRET


## Get the top 100 venues that are in each neighborhood within a radius of 500 meters.

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [7]:
#create a new dataframe called athens_venues
athens_venues = getNearbyVenues(names =df_data_0['Neighborhood'],
                                   latitudes = df_data_0['Latitude'],
                                   longitudes = df_data_0['Longitude']
                                  )
                                  
athens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akademia Platonos,37.991368,23.708381,Τα Σκαλάκια,37.989256,23.709045,Greek Restaurant
1,Akademia Platonos,37.991368,23.708381,Αρχαιολογικό Πάρκο Ακαδημίας Πλάτωνος,37.992353,23.708943,Park
2,Akademia Platonos,37.991368,23.708381,Το Καφενείο Στην Ακαδημία Πλάτωνος,37.993886,23.707794,Kafenio
3,Akademia Platonos,37.991368,23.708381,ΔΙΟΤΙΜΑ Γλυκίσματα,37.989742,23.712603,Dessert Shop
4,Akademia Platonos,37.991368,23.708381,Γρηγόρης,37.987474,23.707624,Snack Place


In [8]:
#Count how many venues were returned for each neighborhood
athens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Akademia Platonos,20,20,20,20,20,20
Ampelokipoi,74,74,74,74,74,74
Ano Kypseli,16,16,16,16,16,16
Ano Petralona,31,31,31,31,31,31
Exarcheia,63,63,63,63,63,63
Gazi,24,24,24,24,24,24
Goudi,93,93,93,93,93,93
Gyzi,87,87,87,87,87,87
Historic Triangle,87,87,87,87,87,87


In [9]:
# check the unique categories
print('There are {} uniques categories.'.format(len(athens_venues['Venue Category'].unique())))

There are 206 uniques categories.


## Analyze each neighborhood

In [10]:
#We first convert Venue Category variable into dummy variables.
# one hot encoding
athens_onehot = pd.get_dummies(athens_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
athens_onehot['Neighborhood'] = athens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [athens_onehot.columns[-1]] + list(athens_onehot.columns[:-1])
athens_onehot = athens_onehot[fixed_columns]

athens_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bistro,Bookstore,Bougatsa Shop,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Department Store,Dessert Shop,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Lingerie Store,Lounge,Magirio,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Office,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Ouzeri,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Souvenir Shop,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Toy / Game Store,Track Stadium,Trail,Tsipouro Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Akademia Platonos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Akademia Platonos,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Akademia Platonos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [11]:
#Group rows by neighborhood and by taking the mean of the frequency of occurrence of each Venue Category.

athens_grouped = athens_onehot.groupby('Neighborhood').mean().reset_index()
athens_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bistro,Bookstore,Bougatsa Shop,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Department Store,Dessert Shop,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Lingerie Store,Lounge,Magirio,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Office,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Ouzeri,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Souvenir Shop,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Toy / Game Store,Track Stadium,Trail,Tsipouro Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Akademia Platonos,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.1,0.050000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.200000,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.050000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.050000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.100000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.05,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.050000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.050000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0000,0.0,0.050000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.100000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.05,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0

In [38]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in athens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = athens_grouped[athens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akademia Platonos----
              venue  freq
0       Coffee Shop   0.2
1          Bus Stop   0.1
2  Greek Restaurant   0.1
3       Supermarket   0.1
4       Snack Place   0.1


----Ampelokipoi----
              venue  freq
0              Café  0.14
1       Coffee Shop  0.09
2            Bakery  0.07
3      Dance Studio  0.05
4  Greek Restaurant  0.05


----Ano Kypseli----
                  venue  freq
0           Supermarket  0.12
1                 Plaza  0.06
2                Bakery  0.06
3          Dessert Shop  0.06
4  Fast Food Restaurant  0.06


----Ano Petralona----
             venue  freq
0             Café  0.16
1              Bar  0.10
2     Cocktail Bar  0.10
3          Kafenio  0.06
4  Meze Restaurant  0.06


----Exarcheia----
          venue  freq
0           Bar  0.14
1   Coffee Shop  0.05
2  Dessert Shop  0.05
3          Café  0.05
4       Magirio  0.03


----Gazi----
             venue  freq
0        Nightclub  0.21
1  Motorcycle Shop  0.08
2              Bar  0.

In [12]:
#Create a new dataframe return_most_common_venues and display the top 10 venues for each neighborhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        # append 'st', 'nd', 'rd' to the top 3 venues
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = athens_grouped['Neighborhood']

for ind in np.arange(athens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(athens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akademia Platonos,Coffee Shop,Snack Place,Greek Restaurant,Supermarket,Bus Stop,Fast Food Restaurant,Kafenio,Toy / Game Store,Café,Hardware Store
1,Ampelokipoi,Café,Coffee Shop,Bakery,Dance Studio,Greek Restaurant,Mobile Phone Shop,Supermarket,Movie Theater,Gym,Snack Place
2,Ano Kypseli,Supermarket,Bar,Fast Food Restaurant,Mobile Phone Shop,Souvlaki Shop,Mountain,Greek Restaurant,Snack Place,Bakery,Soccer Field
3,Ano Petralona,Café,Bar,Cocktail Bar,Kafenio,Meze Restaurant,Coffee Shop,Cupcake Shop,Supermarket,Pub,Cretan Restaurant
4,Exarcheia,Bar,Dessert Shop,Café,Coffee Shop,Theater,Greek Restaurant,Bookstore,Magirio,Vegetarian / Vegan Restaurant,Hotel


## Cluster Neighborhoods

In [14]:
# Run k-means to cluster the neighborhood into 5 clusters. k-means will then partition our neighborhoods into 5 groups. 
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
athens_grouped_clustering = athens_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(athens_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 4, 4, 1, 4, 4, 1], dtype=int32)

In [17]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

athens_merged = df_data_0
athens_merged = athens_merged.merge(neighborhoods_venues_sorted, on = 'Neighborhood')

athens_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akademia Platonos,37.991368,23.708381,1,Coffee Shop,Snack Place,Greek Restaurant,Supermarket,Bus Stop,Fast Food Restaurant,Kafenio,Toy / Game Store,Café,Hardware Store
1,Ampelokipoi,37.989403,23.761144,1,Café,Coffee Shop,Bakery,Dance Studio,Greek Restaurant,Mobile Phone Shop,Supermarket,Movie Theater,Gym,Snack Place
2,Ano Kypseli,38.005780,23.745380,1,Supermarket,Bar,Fast Food Restaurant,Mobile Phone Shop,Souvlaki Shop,Mountain,Greek Restaurant,Snack Place,Bakery,Soccer Field
3,Ano Petralona,37.970467,23.712938,4,Café,Bar,Cocktail Bar,Kafenio,Meze Restaurant,Coffee Shop,Cupcake Shop,Supermarket,Pub,Cretan Restaurant
4,Exarcheia,37.985992,23.735541,4,Bar,Dessert Shop,Café,Coffee Shop,Theater,Greek Restaurant,Bookstore,Magirio,Vegetarian / Vegan Restaurant,Hotel


In [24]:
#visualize the resulting clusters
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(athens_merged['Latitude'], athens_merged['Longitude'], athens_merged['Neighborhood'], athens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [35]:
#cluster1
athens_merged.loc[athens_merged['Cluster Labels'] == 0, athens_merged.columns[[0,1,2,3,4,5,6,7,8,9,10] + list(range(5, athens_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Kolokynthou,37.996749,23.694886,0,Daycare,Auto Garage,Furniture / Home Store,Tennis Court,Other Repair Shop,Rental Car Location,Bougatsa Shop,Auto Garage,Furniture / Home Store,Tennis Court,Other Repair Shop,Rental Car Location,Bougatsa Shop,Hunting Supply,Fish Taverna,Department Store


In [34]:
#cluster2
athens_merged.loc[athens_merged['Cluster Labels'] == 1, athens_merged.columns[[0,1,2,3,4,5,6,7,8,9,10] + list(range(5, athens_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akademia Platonos,37.991368,23.708381,1,Coffee Shop,Snack Place,Greek Restaurant,Supermarket,Bus Stop,Fast Food Restaurant,Kafenio,Snack Place,Greek Restaurant,Supermarket,Bus Stop,Fast Food Restaurant,Kafenio,Toy / Game Store,Café,Hardware Store
1,Ampelokipoi,37.989403,23.761144,1,Café,Coffee Shop,Bakery,Dance Studio,Greek Restaurant,Mobile Phone Shop,Supermarket,Coffee Shop,Bakery,Dance Studio,Greek Restaurant,Mobile Phone Shop,Supermarket,Movie Theater,Gym,Snack Place
2,Ano Kypseli,38.005780,23.745380,1,Supermarket,Bar,Fast Food Restaurant,Mobile Phone Shop,Souvlaki Shop,Mountain,Greek Restaurant,Bar,Fast Food Restaurant,Mobile Phone Shop,Souvlaki Shop,Mountain,Greek Restaurant,Snack Place,Bakery,Soccer Field
6,Goudi,37.984773,23.766414,1,Café,Coffee Shop,Snack Place,Bakery,Supermarket,Greek Restaurant,Souvlaki Shop,Coffee Shop,Snack Place,Bakery,Supermarket,Greek Restaurant,Souvlaki Shop,Burger Joint,Meze Restaurant,Theater
9,Ilissia,37.974863,23.744353,1,Café,Italian Restaurant,Dessert Shop,Bookstore,Cocktail Bar,Bar,Museum,Italian Restaurant,Dessert Shop,Bookstore,Cocktail Bar,Bar,Museum,Coffee Shop,Plaza,History Museum
11,Kato Patissia,38.011783,23.728601,1,Café,Supermarket,Gym / Fitness Center,Snack Place,Mobile Phone Shop,Bakery,Coffee Shop,Supermarket,Gym / Fitness Center,Snack Place,Mobile Phone Shop,Bakery,Coffee Shop,Italian Restaurant,Burger Joint,Movie Theater
15,Kolonos-Liossion-Lenorman,37.996069,23.716221,1,Supermarket,Greek Restaurant,Café,Plaza,Concert Hall,Bougatsa Shop,Park,Greek Restaurant,Café,Plaza,Concert Hall,Bougatsa Shop,Park,Gift Shop,Fast Food Restaurant,Lounge
17,Kypseli,38.002495,23.740768,1,Café,Greek Restaurant,Souvlaki Shop,Dessert Shop,Supermarket,Bar,Gym / Fitness Center,Greek Restaurant,Souvlaki Shop,Dessert Shop,Supermarket,Bar,Gym / Fitness Center,Coffee Shop,Pub,Fish Taverna
21,Nea Kypseli,38.000247,23.746697,1,Café,Bakery,Snack Place,Supermarket,Pizza Place,Park,Cupcake Shop,Bakery,Snack Place,Supermarket,Pizza Place,Park,Cupcake Shop,Donut Shop,Plaza,Coffee Shop
23,Neos Kosmos,37.957581,23.728329,1,Bakery,Souvlaki Shop,Pharmacy,Dessert Shop,Sandwich Place,Greek Restaurant,Café,Souvlaki Shop,Pharmacy,Dessert Shop,Sandwich Place,Greek Restaurant,Café,Meze Restaurant,Snack Place,Coffee Shop


In [33]:
#cluster3
athens_merged.loc[athens_merged['Cluster Labels'] == 2, athens_merged.columns[[0,1,2,3,4,5,6,7,8,9,10] + list(range(5, athens_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Votanicos,37.983514,23.6997,2,Mobile Phone Shop,Print Shop,Women's Store,Fish Market,Frozen Yogurt Shop,French Restaurant,Food Truck,Print Shop,Women's Store,Fish Market,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Stand,Food & Drink Shop,Flower Shop


In [32]:
#cluster4
athens_merged.loc[athens_merged['Cluster Labels'] == 3, athens_merged.columns[[0,1,2,3,4,5,6,7,8,9,10] + list(range(5, athens_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Kaissariani,37.958162,23.799005,3,Mountain,Beach,Trail,Women's Store,Fish Taverna,Furniture / Home Store,Frozen Yogurt Shop,Beach,Trail,Women's Store,Fish Taverna,Furniture / Home Store,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Stand


In [31]:
#cluster5
athens_merged.loc[athens_merged['Cluster Labels'] == 4, athens_merged.columns[[0,1,2,3,4,5,6,7,8,9,10] + list(range(5, athens_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Ano Petralona,37.970467,23.712938,4,Café,Bar,Cocktail Bar,Kafenio,Meze Restaurant,Coffee Shop,Cupcake Shop,Bar,Cocktail Bar,Kafenio,Meze Restaurant,Coffee Shop,Cupcake Shop,Supermarket,Pub,Cretan Restaurant
4,Exarcheia,37.985992,23.735541,4,Bar,Dessert Shop,Café,Coffee Shop,Theater,Greek Restaurant,Bookstore,Dessert Shop,Café,Coffee Shop,Theater,Greek Restaurant,Bookstore,Magirio,Vegetarian / Vegan Restaurant,Hotel
5,Gazi,37.974813,23.702751,4,Nightclub,Bar,Motorcycle Shop,History Museum,Pharmacy,Pool,Café,Bar,Motorcycle Shop,History Museum,Pharmacy,Pool,Café,Snack Place,Furniture / Home Store,Mediterranean Restaurant
7,Gyzi,37.979450,23.716220,4,Bar,Nightclub,Greek Restaurant,Theater,Meze Restaurant,Restaurant,Café,Nightclub,Greek Restaurant,Theater,Meze Restaurant,Restaurant,Café,Cocktail Bar,Mediterranean Restaurant,Dance Studio
8,Historic Triangle,37.979450,23.716220,4,Bar,Nightclub,Greek Restaurant,Theater,Meze Restaurant,Restaurant,Café,Nightclub,Greek Restaurant,Theater,Meze Restaurant,Restaurant,Café,Cocktail Bar,Mediterranean Restaurant,Dance Studio
12,Kato Petralona,37.971826,23.708129,4,Café,Bar,Nightclub,Coffee Shop,Greek Restaurant,Cocktail Bar,Pastry Shop,Bar,Nightclub,Coffee Shop,Greek Restaurant,Cocktail Bar,Pastry Shop,Pool,Plaza,Cretan Restaurant
14,Kolonaki,37.986653,23.734839,4,Bar,Coffee Shop,Dessert Shop,Hotel,Magirio,Greek Restaurant,Café,Coffee Shop,Dessert Shop,Hotel,Magirio,Greek Restaurant,Café,Souvlaki Shop,Vegetarian / Vegan Restaurant,Meze Restaurant
16,Koukaki,37.963456,23.722023,4,Café,Kafenio,Pizza Place,Bar,Hotel,Coffee Shop,Cosmetics Shop,Kafenio,Pizza Place,Bar,Hotel,Coffee Shop,Cosmetics Shop,Dessert Shop,Pedestrian Plaza,Dance Studio
18,Metaxourgeio,37.983150,23.718824,4,Theater,Greek Restaurant,Hotel,Bar,Mediterranean Restaurant,Café,Kafenio,Greek Restaurant,Hotel,Bar,Mediterranean Restaurant,Café,Kafenio,Meze Restaurant,Restaurant,Souvlaki Shop
19,Monastiraki,37.976101,23.725651,4,Bar,Historic Site,Café,Dessert Shop,Ice Cream Shop,Greek Restaurant,Plaza,Historic Site,Café,Dessert Shop,Ice Cream Shop,Greek Restaurant,Plaza,Gift Shop,Indian Restaurant,Wine Bar
